# Setup

In [7]:
# Load Libraries:

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib.image as mpimg
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os
import h5py
import math
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


# Create ResNet Utilities

In [8]:
# Some of these functions might or might not be used in this particular model and dataset. 
# However, they are still defined just in case they are needed for future model building. 

def load_dataset():
    train_dataset = h5py.File('datasets/signs_mnist/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/signs_mnist/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples) (m, n_y)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches


def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


def forward_propagation_for_predict(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3'] 
                                                           # Numpy Equivalents:
    Z1 = tf.add(tf.matmul(W1, X), b1)                      # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                    # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)                     # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                    # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)                     # Z3 = np.dot(W3,Z2) + b3
    
    return Z3

def predict(X, parameters):
    
    W1 = tf.convert_to_tensor(parameters["W1"])
    b1 = tf.convert_to_tensor(parameters["b1"])
    W2 = tf.convert_to_tensor(parameters["W2"])
    b2 = tf.convert_to_tensor(parameters["b2"])
    W3 = tf.convert_to_tensor(parameters["W3"])
    b3 = tf.convert_to_tensor(parameters["b3"])
    
    params = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3}
    
    x = tf.placeholder("float", [12288, 1])
    
    z3 = forward_propagation_for_predict(x, params)
    p = tf.argmax(z3)
    
    sess = tf.Session()
    prediction = sess.run(p, feed_dict = {x: X})
        
    return prediction


# Construction Phase of Residual Deep Neural Network

In [9]:
# First we create the building blocks to be used in the construction of the ResNet.

# There are two types of blocks that can be constructed:
    # The IDENTITY BLOCK is used when the matrices for the inputs and outputs are of equal size.
    # The CONVOLUTIONAL BLOCK is used when the matrices for the inputs and outputs are different dimensions.
        # To make the matrices the same, there has to be a passthrough layer.        

In [10]:
# First, let's create the identity block:

def identity_block(X, f, filters, stage, block):
    """
    
    Implementation of the identity block:
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of the shape (n_H, n_W, n_C)
    """
    
    # defining name basis:
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters:
    F1, F2, F3 = filters
    
    # Save the input value:
    X_shortcut = X
    
    #####MAIN PATH#####
    # First component of the main path:
    X = Conv2D(filters = F1, kernel_size = (1,1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X) 
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path (approximately 3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    # Third component of main path ( approximately 2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
    
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (approximately 2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X
    

In [11]:
# Next, let's create the convolutional block:

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + 'branch'
    bn_name_base = 'bn' + str(stage) + block + 'branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X
    
    
    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X
    

In [12]:
# So now we will build the actual 50 layer ResNet model:

def ResNet50(input_shape=(256, 256, 3), classes=3):
    """
    Implementation of the popular ResNet50 using the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 
    -> CONVBLOCK -> CK*3 -> CK*3 -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK 
    -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER
    
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes
    
    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    
    ### START CODE HERE ###
    
    # Stage 3 (Approximately 4 lines)
    X = convolutional_block(X, f=3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    
    # Stage 4 (Approximately 6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2 )
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    
    # Stage 5 (Approximately 3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    
    # AVGPOOL (Approximately 1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), name="avg_pool")(X)
    
    ### END CODE HERE ###
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    
    return model



In [13]:
# Now we run the following code to build the model's graph. 
# If the implementation is not correct, we will know it by checking the accuracy when running model.fit(...)

model = ResNet50(input_shape = (256, 256, 3), classes = 3)

In [14]:
# As seen in the Keras Tutorial Notebook, prior to training a model, you need to configure the learning process by compiling the model.

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Execution Phase

In [15]:
# During the execution phase we have to do the following:
    # First, upload the training data.
    # Scale the data (between 0 and 1)
    # Shuffles the data
    # Build a function to load one minibatch at a time.

# Load Dataset and Create Image Augmentation Pipeline

In [21]:
# Featurewise_Centering:
    # Scale pixel values have a zero mean.
    # The method involves calculating the mean pixel value across the entire training dataset, then subtracting it from each image.
    # This centers the distribution of pixel values on zero, meaning the pixel value for centered images will be zero.
    # The point of this is to mathematically decrease the variance and variety of the data allowing the machine learning model to learn more accurately.
    
    
# Featurewise Standard Normalization:
    # In image processing, Normalization is a process that changes the range of pixel intensity values. 
    # Applications include photographs with poor contrast due to glare, making a newspaper have a similar range of grayscale to avoid distraction or fatigue, or bringing the image or other type of signal into a range that is more familiar or normal to the senses.
    # In our case of computer vision, our motivation is to achieve consistency and a smaller dynamic range to allow the Machine learning model to more easily learn on the data.
    # This is the same principle as resizing images so that they are a similar size so as to make it easier for the model to learn more accurately.
    # Standardizing the Normalization process means that features will be rescaled so that they'll have properties of a standard normal distribution between 0 and 1. 
    # Pixel Standardization is supported at two levels: either per-image(sample-wise) or per dataset(feature-wise)

# In order to implement Featurewise Centering and Standardized Normalization
    # You first have to reshape the dataset to have 1 channel instead of 3.
    # To reduce channels means to change an image from RGB (3 channel) to Grayscale (1 channel). 
    
# Next you want to reshape the images to be the same dimensions [samples][width][height][channels].
    # The reason we do this is because these are the dimensions that the Resnet-50 expects.
    # In this case the dimensions that are expected are: [height, width, # of channels, number of classes]

# Rescale is a value by which we will multiply the data before any other processing. 
    # Our original images consist of RGB values between 0-255, but such values would be too high for our model to process (given a typical learning rate).
    # So we target values between 0 and 1 instead by scaling with a 1/255 factor.
    
# We have to convert image to numpy array.

#

# Define Image Augmentation parameters:

featurewise_center=False
featurewise_std_normalization=False
rescale= 1./255
rotation_range=45
height_shift_range=.15
width_shift_range=.15
horizontal_flip=True
zoom_range=0.5

# Define flow_from_directory parameters:
train_dir = '/home/lam/ml/datasets/Psychedelics/Train'
val_dir = '/home/lam/ml/datasets/Psychedelics/Validation/'
test_dir = '/home/lam/ml/datasets/Psychedelics/Test/'
class_mode = 'categorical'
batch_size = 32
shuffle = True
IMG_HEIGHT = 256
IMG_WIDTH = 256
save_to_dir= '/home/lam/ml/datasets/Psychedelics/Augmented'
save_prefix='aug'
save_format='jpg'



# Define Train, Validation and Test Image Pipeline:
datagen = ImageDataGenerator(
                             rescale=rescale,
                             rotation_range=rotation_range,
                             width_shift_range=width_shift_range,
                             height_shift_range=height_shift_range,
                             horizontal_flip=horizontal_flip,
                             zoom_range=zoom_range
                             )


#load and iterate training dataset:
train_it = datagen_train.flow_from_directory(directory=train_dir, 
                                             class_mode=class_mode, 
                                             batch_size=batch_size,
                                             shuffle=shuffle,
                                             target_size=(IMG_HEIGHT, IMG_WIDTH),
                                             color_mode='rgb'
                                             )
                            

# load and iterate validation dataset:
val_it = datagen.flow_from_directory(directory=val_dir, 
                                     class_mode=class_mode, 
                                     batch_size=batch_size,
                                     shuffle=shuffle,
                                     target_size=(IMG_HEIGHT, IMG_WIDTH)
                                     color_mode='rgb'
                                     )

# load and iterate test dataset:
test_it = datagen.flow_from_directory(directory=test_dir, 
                                      class_mode=class_mode, 
                                      batch_size=batch_size,
                                      shuffle=shuffle,
                                      target_size=(IMG_HEIGHT, IMG_WIDTH)
                                      )


Found 1558 images belonging to 3 classes.
Found 220 images belonging to 3 classes.
Found 205 images belonging to 3 classes.


In [ ]:
# Now we run the following code to train the model on 759 epochs with a batch size of 32.
model.fit_generator(train_it, steps_per_epoch = 64)

Epoch 1/1


In [15]:
# Let's see how this model performs on the test set:

preds = model.evaluate(test_it)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))

NameError: name 'X_test' is not defined

In [16]:
img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)
my_image = scipy.misc.imread(img_path)
imshow(my_image)
print("class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ")
print(model.predict(x))

FileNotFoundError: [Errno 2] No such file or directory: 'images/my_image.jpg'

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [ ]:
plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))